In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from PIL import ImageFile

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Lambda,Dense,GlobalAveragePooling1D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization,ConvLSTM2D
from tensorflow.keras.layers import MaxPooling2D,LSTM,GlobalMaxPool1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

import tensorflow as tf
import cv2
import numpy
import gc
import keras
import pandas as pd
import numpy as np
import os

In [ ]:
import pandas as pd
import numpy as np



In [ ]:
csv = pd.read_csv("training-labels.csv")

In [ ]:
csv

,Filename,Drscore
0,train_image2.jpeg,0
1,train_image3.jpeg,0
2,train_image4.jpeg,0
3,train_image5.jpeg,0
4,train_image6.jpeg,0
...,...,...
14140,train_image14142.jpg,2
14141,train_image14143.jpg,2
14142,train_image14144.jpg,2
14143,train_image14145.jpg,2


A kódrészlet a következőket teszi:

* A `csv.values.tolist()` függvény a CSV fájlt egy listába konvertálja, ahol minden elem egy 2 elemű tömb, az első elem az image_path, a második elem pedig a label.
* A `adatok.append()` függvény minden tömböt hozzáad az `adatok` listához.
* A `adatok = pd.DataFrame(adatok,columns=[...])` függvény a `adatok` listát egy Pandas dataframe-be konvertálja, ahol a következő oszlopok vannak:
    * `adaptivbinarizalt`: az adaptív binarisálással feldolgozott kép
    * `cany10`: a Canny edge detektálással feldolgozott kép, 10 értékű sigma paraméterrel
    * `gradeient`: a gradient detektálással feldolgozott kép
    * `szurkitet`: a szurkítással feldolgozott kép
    * `adaptivbinarizaltblur`: az adaptív binarisálással feldolgozott kép, blurrel
    * `cany100`: a Canny edge detektálással feldolgozott kép, 100 értékű sigma paraméterrel
    * `laplace`: a Laplacian detektálással feldolgozott kép
    * `ujrameretezet`: az újraméretezéssel feldolgozott kép
    * `dilatacio`: a dilatációval feldolgozott kép
    * `nyitas`: a nyítással feldolgozott kép
    * `zaras`: a zárással feldolgozott kép
    * `binarizalttor`: a binarisálással és a torzítással feldolgozott kép
    * `erozio`: az erózióval feldolgozott kép
    * `binarizalttorinv`: a binarisálással és az inverz torzítással feldolgozott kép
    * `label`: az eredeti kép osztálya

A dataframe-et ezután felhasználhatjuk további elemzéshez, például gépi tanuláshoz.

In [ ]:
adatok = []
for i in csv.values.tolist():
    adatok.append(["adaptivbinarizalt/" +i[0]   ,
                   "cany10/"+i[0]  ,
                   "gradeient/"+i[0]  ,
                   "szurkitet/"+i[0]  ,
                   "adaptivbinarizaltblur/"+i[0]  ,
                   "cany100/"+i[0]  ,
                   "laplace/"+i[0]  ,
                   "ujrameretezet/"+i[0]  ,
                   "dilatacio/"+i[0]  ,
                   "nyitas/"+i[0]  ,
                   "zaras/"+i[0]  ,
                   "binarizalttor/"+i[0]  ,
                   "erozio/"+i[0]  ,
                   "binarizalttorinv/"+i[0]  ,
                   i[1]])

adatok = pd.DataFrame(adatok,columns= ["adaptivbinarizalt",
                                       "cany10",
                                       "gradeient",
                                       "szurkitet",
                                       "adaptivbinarizaltblur",
                                       "cany100",
                                       "laplace",
                                       "ujrameretezet",
                                       "dilatacio",
                                       "nyitas",
                                       "zaras",
                                       "binarizalttor",
                                       "erozio",
                                       "binarizalttorinv",
                                       "label"])

In [ ]:
import tensorflow as tf

The Python code you provided is used to create two TensorFlow Keras ImageDataGenerators, one for training and one for validation. The ImageDataGenerator class is used to generate batches of image data from a directory or a Pandas DataFrame.

The first ImageDataGenerator, `datagen`, is created with the `validation_split=0.25` argument, which specifies that 25% of the data should be used for validation. The second ImageDataGenerator, `datagenadaptiv`, is created with the same arguments.

The `train_generator` is created using the `datagen.flow_from_dataframe()` method. This method takes the following arguments:

* `dataframe`: A Pandas DataFrame containing the image paths and labels.
* `directory`: The directory containing the images.
* `x_col`: The name of the column in the DataFrame containing the image paths.
* `y_col`: The name of the column in the DataFrame containing the labels.
* `subset`: The subset of the data to use, either "training" or "validation".
* `classes`: A list of the class labels.
* `batch_size`: The size of the batches to generate.
* `seed`: A random seed to use for shuffling the data.
* `shuffle`: A Boolean value specifying whether to shuffle the data.
* `class_mode`: The type of class labels to output, either "categorical" or "sparse".
* `target_size`: The size to resize the images to.

The `valid_generator` is created using the same arguments, except that the `subset` argument is set to "validation".

The `train_generator` and `valid_generator` can then be used to train and evaluate a TensorFlow Keras model.

In [ ]:
#adatok = pd.read_csv("training-labels.csv")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
datagen=ImageDataGenerator(validation_split=0.25)
adatok["label"]= adatok["label"].map(str)
datagenadaptiv=ImageDataGenerator(validation_split=0.25)
train_generator=datagen.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="dilatacio",
y_col="label",
subset="training",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
valid_generator=datagen.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="dilatacio",
y_col="label",
subset="validation",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))


Found 0 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 14145 invalid image filename(s) in x_col="dilatacio". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames belonging to 5 classes.


The `valid_generator.next()` function in TensorFlow Keras returns the next batch of validation data. The batch of data is a tuple, where the first element is a NumPy array containing the images and the second element is a NumPy array containing the labels.

The images in the batch are resized to the `target_size` specified when the ImageDataGenerator was created. The labels are converted to the `class_mode` specified when the ImageDataGenerator was created.

The `valid_generator.next()` function can be used in a for loop to iterate over the validation data. For example, the following code snippet shows how to iterate over the validation data and evaluate a TensorFlow Keras model

In [ ]:
valid_generator.next()

(array([], shape=(0, 128, 128, 3), dtype=float32), array([], dtype=float32))

In [ ]:
adatok.loc[:,["adaptivbinarizalt","cany10","cany100","erozio","dilatacio","label"]]

,adaptivbinarizalt,cany10,cany100,erozio,dilatacio,label
0,adaptivbinarizalt/train_image2.jpeg,cany10/train_image2.jpeg,cany100/train_image2.jpeg,erozio/train_image2.jpeg,dilatacio/train_image2.jpeg,0
1,adaptivbinarizalt/train_image3.jpeg,cany10/train_image3.jpeg,cany100/train_image3.jpeg,erozio/train_image3.jpeg,dilatacio/train_image3.jpeg,0
2,adaptivbinarizalt/train_image4.jpeg,cany10/train_image4.jpeg,cany100/train_image4.jpeg,erozio/train_image4.jpeg,dilatacio/train_image4.jpeg,0
3,adaptivbinarizalt/train_image5.jpeg,cany10/train_image5.jpeg,cany100/train_image5.jpeg,erozio/train_image5.jpeg,dilatacio/train_image5.jpeg,0
4,adaptivbinarizalt/train_image6.jpeg,cany10/train_image6.jpeg,cany100/train_image6.jpeg,erozio/train_image6.jpeg,dilatacio/train_image6.jpeg,0
...,...,...,...,...,...,...
14140,adaptivbinarizalt/train_image14142.jpg,cany10/train_image14142.jpg,cany100/train_image14142.jpg,erozio/train_image14142.jpg,dilatacio/train_image14142.jpg,2
14141,adaptivbinarizalt/train_image14143.jpg,cany10/train_image14143.jpg,cany100/train_image14143.jpg,erozio/train_image14143.jpg,dilatacio/train_image14143.jpg,2
14142,adaptivbinarizalt/train_image14144.jpg,cany10/train_image14144.jpg,cany100/train_image14144.jpg,erozio/train_image14144.jpg,dilatacio/train_image14144.jpg,2
14143,adaptivbinarizalt/train_image14145.jpg,cany10/train_image14145.jpg,cany100/train_image14145.jpg,erozio/train_image14145.jpg,dilatacio/train_image14145.jpg,2


The Python function `generate_generator_multiple()` takes two arguments:

* `generator`: A TensorFlow Keras ImageDataGenerator object.
* `adatok`: A Pandas DataFrame containing the image paths and labels.

The function creates five ImageDataGenerators, one for each of the following image transformations:

* `adaptivbinarizaltblur`: Adaptive thresholding with blur.
* `cany10`: Canny edge detection with sigma=10.
* `dilatacio`: Dilation.
* `erozio`: Erosion.
* `cany100`: Canny edge detection with sigma=100.

The function then creates a while loop that yields batches of images and labels from the five ImageDataGenerators. The batches are returned as a tuple, where the first element is a list of NumPy arrays containing the images and the second element is a NumPy array containing the labels.

The `generate_generator_multiple()` function can be used to train a TensorFlow Keras model on multiple image transformations.

The `generate_generator_multiple()` function can also be used to perform data augmentation on training data. Data augmentation is a technique that can improve the performance of machine learning models by increasing the size and diversity of the training dataset.

To perform data augmentation, simply create the `generate_generator_multiple()` function with the `ImageDataGenerator` objects that contain the desired image transformations. The function will then yield batches of augmented images and labels.

In [ ]:
def generate_generator_multiple(generator,adatok):
    #datagenadaptiv=ImageDataGenerator(validation_split=0.25)
    train_generator4=generator.flow_from_dataframe(
    dataframe=adatok,
    directory="./atalakitot128/",
    x_col="cany10",
    y_col="label",
    subset="training",
    classes=['0','1','2','3','4'],
    batch_size=128,
    seed=42,
    shuffle=False,
    class_mode="sparse",
    target_size=(128, 128))
    train_generator=generator.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="adaptivbinarizaltblur",
y_col="label",
subset="training",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    train_generator3=generator.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="dilatacio",
y_col="label",
subset="training",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    train_generator2=generator.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="erozio",
y_col="label",
subset="training",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    train_generator5=datagen.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="cany100",
y_col="label",
subset="training",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    while True:
            X1i = train_generator.next()
            X2i = train_generator2.next()
            X3i = train_generator3.next()
            X4i = train_generator4.next()
            X5i = train_generator5.next()
            yield [X1i[0], X2i[0],X3i[0],X4i[0],X5i[0]], X2i[1]  #Yield both images and their mutual label

In [ ]:
def generate_generator_multiple_test(generator,adatok):
    #datagenadaptiv=ImageDataGenerator(validation_split=0.25)
    train_generator4=generator.flow_from_dataframe(
    dataframe=adatok,
    directory="./atalakitot128/",
    x_col="cany10",
    y_col="label",
    subset="validation",
    classes=['0','1','2','3','4'],
    batch_size=128,
    seed=42,
    shuffle=False,
    class_mode="sparse",
    target_size=(128, 128))
    train_generator=generator.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="adaptivbinarizaltblur",
y_col="label",
subset="validation",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    train_generator3=generator.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="dilatacio",
y_col="label",
subset="validation",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    train_generator2=generator.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="erozio",
y_col="label",
subset="validation",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    train_generator5=datagen.flow_from_dataframe(
dataframe=adatok,
directory="./atalakitot128/",
x_col="cany100",
y_col="label",
subset="validation",
classes=['0','1','2','3','4'],
batch_size=128,
seed=42,
shuffle=False,
class_mode="sparse",
target_size=(128, 128))
    while True:
            X1i = train_generator.next()
            X2i = train_generator2.next()
            X3i = train_generator3.next()
            X4i = train_generator4.next()
            X5i = train_generator5.next()
            yield [X1i[0], X2i[0],X3i[0],X4i[0],X5i[0]], X2i[1]  #Yield both images and their mutual label

In [ ]:
#adatok = pd.read_csv("training-labels.csv")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
datagen=ImageDataGenerator(validation_split=0.25)
adatok["label"]= adatok["label"].map(str)

In [ ]:
train = generate_generator_multiple(datagen,adatok)
test = generate_generator_multiple_test(datagen,adatok)

In [ ]:
next(iter(train))

Found 0 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 14145 invalid image filename(s) in x_col="cany10". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 14145 invalid image filename(s) in x_col="adaptivbinarizaltblur". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames belonging to 5 classes.


...

Found 0 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 14145 invalid image filename(s) in x_col="erozio". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 14145 invalid image filename(s) in x_col="cany100". These filename(s) will be ignored.
  warnings.warn(


([array([], shape=(0, 128, 128, 3), dtype=float32),
  array([], shape=(0, 128, 128, 3), dtype=float32),
  array([], shape=(0, 128, 128, 3), dtype=float32),
  array([], shape=(0, 128, 128, 3), dtype=float32),
  array([], shape=(0, 128, 128, 3), dtype=float32)],
 array([], dtype=float32))